In [1]:
from os.path import dirname, join as pjoin
import scipy.io as sio
import numpy as np

In [2]:
mat_fname = pjoin('mesh', 'strip2.mat')

print(mat_fname)

mat_contents = sio.loadmat(mat_fname)

mesh\strip2.mat


In [3]:
p = mat_contents['p']
t = mat_contents['t']

In [4]:
[s1,s2] = p.shape
s1,s2

(3, 234)

In [5]:
if s1 == 2:
    print("Add third dimension")
    p = np.append(p,np.zeros((1,s2)),axis=0) 

In [6]:
p.shape

(3, 234)

In [7]:
TrianglesTotal=t.shape[1]
TrianglesTotal

244

Find areas of separate triangles

In [11]:
Area = np.zeros(TrianglesTotal)
Center = np.zeros((3,TrianglesTotal))
for m in range(TrianglesTotal):
        N = t[0:3,m]-1
        #print(N)
        Vec1 = p[:,N[0]] - p[:,N[1]]
        Vec2 = p[:,N[2]] - p[:,N[1]]
        Area[m] = np.linalg.norm(np.cross(Vec1,Vec2))/2
        Center[:,m]=1/3*sum(p[:,N],1)

In [12]:
print(Area.shape)
print(Center.shape)

(244,)
(3, 244)


Find all edge elements "Edge_" with at least two adjacent triangles

In [86]:
n = 0
Edge_ = []
TrianglePlus = []
TriangleMinus = []
for m in range(TrianglesTotal):
    N = t[0:3,m] - 1 
    for k in range(m+1,TrianglesTotal):
        M = t[0:3,k] - 1      
        a = 1 - np.array([1 if i else 0 for i in [all(N-M[0]),all(N-M[1]),all(N-M[2])]])
        if(sum(a)==2): #triangles m and k have common edge
            Edge_.append([M[i] for i,j in enumerate(a) if j]); 
            TrianglePlus.append(m);
            TriangleMinus.append(k); 
            n = n + 1;

In [44]:
N = t[0:3,0] - 1
M = t[0:3,1] - 1
print(N)
print(M)

[  1   0 233]
[  4   1 233]


In [89]:
Edge_[0]

[1, 233]

In [91]:
EdgesTotal = len(Edge_)
EdgesTotal

255

This block is only meaningful for T junctions. It leaves only two edge elements at a junction

In [ ]:
Edge__ = [Edge_[2,:],Edge_[1,:];
Remove = []
for m=1:EdgesTotal
    Edge_m=repmat(Edge_(:,m),[1 EdgesTotal]);
    Ind1=any(Edge_  -Edge_m);
    Ind2=any(Edge__ -Edge_m);
    A=find(Ind1.*Ind2==0);
    if(length(A)==3)    %three elements formally exist at a junction 
        Out=find(t(4,TrianglePlus(A))==t(4,TriangleMinus(A)));
        Remove=[Remove A(Out)];
    end
end
Edge_(:,Remove)         =[];
TrianglePlus(Remove)    =[];
TriangleMinus(Remove)   =[];
EdgesTotal=length(Edge_)

In [100]:
Edge__ = [[i[1], i[0]] for i in Edge_]
Remove = []

In [ ]:
for m in range(1,EdgesTotal):
    Edge_m = repmat(Edge_(:,m),[1 EdgesTotal]);
    Ind1=any(Edge_  -Edge_m);
    Ind2=any(Edge__ -Edge_m);
    A=find(Ind1.*Ind2==0);
    if(length(A)==3)    %three elements formally exist at a junction 
        Out=find(t(4,TrianglePlus(A))==t(4,TriangleMinus(A)));
        Remove=[Remove A(Out)];
